In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h +1, X.shape[1] - w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [6]:
# 实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self,x):
        return corr2d(x, self.weight) + self.bias

In [7]:
# 一个简单应用：检测图像中不同颜色的边缘
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [10]:
K = torch.tensor([[1.0,-1.0]])
K

tensor([[ 1., -1.]])

输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [11]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只能检测垂直边缘

学习由X生成Y的卷积核

In [12]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)

X = X.reshape((1,1,6,8)) # 通道维、批量大小维
Y = Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad.data
    if (i + 1)%2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 18.290
batch 4, loss 6.268
batch 6, loss 2.362
batch 8, loss 0.933
batch 10, loss 0.376


In [13]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9253, -1.0513]])